# ACLUm BPD Open Checkbook Surveillance Project

We aim to cross-reference BPD's Open Checkbook data against the Surveillance Industry Index.

You can download the data yourself here:

```
$ wget -O ledger.csv https://spending.data.boston.gov/api/ledger.csv
$ wget -O sii.html https://sii.transparencytoolkit.org/
```

### Get the list of surveillance companies

SII doesn't have an API that just grabs the list so we'll regex out the list in the sidebar from the HTML. There are exactly 500 of them, which is a _bit_ suspicious.

In [1]:
import re
import itertools
import urllib
pattern = re.compile(r"company_name_facet=(.*?)&amp")

with open('sii.html', 'r') as sii:
    messy = itertools.chain.from_iterable(map(lambda line: re.findall(pattern, line), sii))
    surveillance_companies = list(map(lambda s: urllib.unquote_plus(s), messy))

### Get the list of BPD vendors

In [2]:
import pandas
ledger = pandas.read_csv("ledger.csv")
#list(ledger)

/usr/lib/python2.7/dist-packages/pandas/io/parsers.py:1139: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
police = ledger.loc[ledger['dept_name'] == "Police Department"]
vendors = map(lambda s: s.lower(), police.vendor_name.unique())

A function to find the common overlap between two strings of text.

In [59]:
def longest_common_substring(s1, s2):
   m = [[0] * (1 + len(s2)) for i in xrange(1 + len(s1))]
   longest, x_longest = 0, 0
   for x in xrange(1, 1 + len(s1)):
       for y in xrange(1, 1 + len(s2)):
           if s1[x - 1] == s2[y - 1]:
               m[x][y] = m[x - 1][y - 1] + 1
               if m[x][y] > longest:
                   longest = m[x][y]
                   x_longest = x
           else:
               m[x][y] = 0
   return s1[x_longest - longest: x_longest]

### Doing the comparison

We'll compare every vendor in the list with the list of companies. if there's more than a 50% length match, we'll mark it as interesting. It turns out that a lot of these companies are called Something **Technologies** or Whatever **Systems** so we'll build a list of these words that can be ignored.

In [87]:
import string
stripwords = ["technologies", "technology",
              "system", "systems",
              "network", "networks",
              "communication", "communications", 
              "electronic", "electronics",
              "service", "services",
              "solution", "solutions",
              "forensic", "forensics",
              "consultant", "consultants",
              "international", "software", "corporation",  "group", "tactical", "security"]
def sanitize(word):
    nopunc = word.translate(None, string.punctuation)
    return " ".join(filter(lambda e: e not in stripwords, nopunc.split()))
vendor_data = map(lambda v: (v, sanitize(v)), vendors)
company_data = map(lambda c: (c, sanitize(c)), surveillance_companies)

In [88]:
matches = []
for (vendor, strip_vendor) in vendor_data:
    for (company, strip_company) in company_data:
        substr = longest_common_substring(strip_vendor, strip_company)
        if len(substr) > (max(len(strip_vendor), len(strip_company)) / 2):
            matches.append({"left": vendor, "right": company})

In [91]:
from csv import DictWriter
with open("surveillance.csv", 'w+') as surveillance:
    dw = DictWriter(surveillance, fieldnames=["left", "right"])
    dw.writeheader()
    for match in matches:
        dw.writerow(match)

### Get the list of purchases from the listed vendors

In [11]:
surveilly_vendors = ["cellebrite usa,corp.", "federal signal corp.", "guidance software",
                     "harris corporation, gcsd", "northrop grumman information technology",
                     "northrop grumman systems corp.", "paraben corporation", "salient stills, inc.",
                     "geofeedia, inc.", "pen link ltd."]

police['vendor_lower'] = police.apply(lambda r: r.vendor_name.lower(), axis=1)
purchases = police[ (police.vendor_lower.isin(surveilly_vendors)) ]

In [12]:
purchases

,voucher,voucher_line,distribution_line,entered,month_number,fiscal_month,month,fiscal_year,year,vendor_name,account,account_descr,dept,dept_name,_6_digit_org_name,monetary_amount,vendor_lower
72365,1754758,1,1,2012-06-01T00:00:00.000,6,12,June,2012,2012,"Cellebrite USA,Corp.",55911,Equipment Other,211100,Police Department,Police Commissioner's Office,4499.00,"cellebrite usa,corp."
72366,1754758,2,1,2012-06-01T00:00:00.000,6,12,June,2012,2012,"Cellebrite USA,Corp.",55911,Equipment Other,211100,Police Department,Police Commissioner's Office,85.00,"cellebrite usa,corp."
72369,1816977,2,1,2012-12-17T00:00:00.000,12,6,December,2013,2012,"Cellebrite USA,Corp.",55911,Equipment Other,211100,Police Department,Police Commissioner's Office,65.00,"cellebrite usa,corp."
72370,1816977,1,1,2012-12-17T00:00:00.000,12,6,December,2013,2012,"Cellebrite USA,Corp.",55911,Equipment Other,211100,Police Department,Police Commissioner's Office,3999.99,"cellebrite usa,corp."
72372,1882690,1,1,2013-05-23T00:00:00.000,5,11,May,2013,2013,"Cellebrite USA,Corp.",54906,Current Software Licen/Maint,211100,Police Department,Police Commissioner's Office,416.25,"cellebrite usa,corp."
72373,1882521,2,1,2013-05-23T00:00:00.000,5,11,May,2013,2013,"Cellebrite USA,Corp.",53909,MiscSup Other,211206,Police Department,TELECOM. MGMT. GROUP,35.00,"cellebrite usa,corp."
72374,1882521,1,1,2013-05-23T00:00:00.000,5,11,May,2013,2013,"Cellebrite USA,Corp.",53909,MiscSup Other,211206,Police Department,TELECOM. MGMT. GROUP,2199.00,"cellebrite usa,corp."
72376,1953000,2,1,2013-11-26T00:00:00.000,11,5,November,2014,2013,"Cellebrite USA,Corp.",54906,Current Software Licen/Maint,211100,Police Department,Police Commissioner's Office,1999.00,"cellebrite usa,corp."
72377,1953000,1,1,2013-11-26T00:00:00.000,11,5,November,2014,2013,"Cellebrite USA,Corp.",54906,Current Software Licen/Maint,211100,Police Department,Police Commissioner's Office,1099.00,"cellebrite usa,corp."
72378,2001337,1,1,2014-04-08T00:00:00.000,4,10,April,2014,2014,"Cellebrite USA,Corp.",55911,Equipment Other,211100,Police Department,Police Commissioner's Office,4990.00,"cellebrite usa,corp."
